<a href="https://colab.research.google.com/github/LC1332/Chat-Haruhi-Suzumiya/blob/main/characters/novel_collecting/system_prompt_suggestion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chat凉宫春日 Chat-Haruhi-Suzumiya

[![Code License](https://img.shields.io/badge/Code%20License-Apache_2.0-green.svg)]()
[![Data License](https://img.shields.io/badge/Data%20License-CC%20By%20NC%204.0-red.svg)]()
[![Huggingface Gradio](https://img.shields.io/static/v1?label=Demo&message=Huggingface%20Gradio&color=orange)](https://huggingface.co/spaces/silk-road/ChatHaruhi)


本脚本是李鲁鲁开发的自动角色总结 脚本

可以争取去总结某个角色的人物个性


---

**Chat凉宫春日**是模仿凉宫春日等一系列动漫人物，使用近似语气、个性和剧情聊天的语言模型，

<details>
  <summary> 本项目由李鲁鲁，冷子昂，闫晨曦，封小洋，scixing，沈骏一，Aria Fei, 米唯实, 吴平宇, 贾曜恺等开发。 </summary>

李鲁鲁发起了项目，并完成了最早的版本，在多个微信群实现了测试。完成了训练数据自动生成对话部分，设计了整体的路线，并撰写报告。

冷子昂负责了每一个阶段的Gradio开发，以及每个部分的功能整合和架构设计。

闫晨曦一开始将李鲁鲁的notebook重构为app.py，参与了WebUI的embedding部分重构整合。

封小洋进行了中文转日文模型的选型，完成了针对台词抽取图片的工具。整合了声纹识别。即将继续参加台本工具的开发。

scixing实践了VITS语音，完成了台词对应的语音抽取，实现了第一个版本的声纹分类。

沈骏一实现了使用ChatGLM2 finetune实验

Aria(Yaying Fei)对接了whisper到台本工具。即将继续参加台本工具的开发。

米唯实实现了Chat哆啦A梦的分支版本

吴平宇部署了ChatGLM2的训练程序，并提供了训练的计算资源。

贾曜恺开发了一个Vue前端实现方案

</details>

## 基础准备

In [1]:
!pip -q install openai tiktoken langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 65.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.6 MB/s eta 0:00:00


In [2]:
import os
import openai

openai.api_key = 'sk-VvF4' # 在这里输入你的OpenAI API Token

os.environ["OPENAI_API_KEY"] = openai.api_key

In [3]:
smart_system_prompt = """You are ChatGPT, a large language model trained by OpenAI.
Knowledge cutoff: 2021-09
Current date: 2023-03-15"""

In [4]:
import tiktoken

enc = tiktoken.get_encoding("cl100k_base")

显然，这个已经有一点大了，我们使用zip包的形式来获取人物

我们假设已经上传了一个linghuchong_text.zip

In [5]:
# !git clone https://github.com/LC1332/Chat-Haruhi-Suzumiya/
!unzip -q linghuchong_text.zip -d /content/linghuchong
!unzip -q yuebuqun_text.zip -d /content/yuebuqun


数据读取和指定角色名

In [6]:
role_name = '令狐冲'
role_name_en = 'linghuchong'
world_name = '笑傲江湖'

max_predict_token = 2500

role_path = '/content/' + role_name_en + '/content/'
role_text_path = role_path + 'texts/'

role_name = 'Dumbledore'

role_path = '/content/Chat-Haruhi-Suzumiya/roles/' + role_name + '/'
role_text_path = role_path + 'texts/'

实现一段python程序，扫描role_text_path对应下的所有txt文件，使用utf8编码读取

保存到一个名为role_texts的list of string中

In [9]:
#@title 定义get_chunks函数
import os
import codecs

import random

def get_chunks(role_text_path, max_predict_token=2500):

    role_texts = []

    for filename in os.listdir(role_text_path):
        if filename.endswith('.txt'):
            with codecs.open(os.path.join(role_text_path, filename), 'r', 'utf-8') as f:
                role_texts.append(f.read())

    random.shuffle(role_texts)

    role_chunk = []
    chunk = ''
    current_len = 0
    for text in role_texts:
        len_text = len(enc.encode(text))
        if current_len + len_text <= max_predict_token:
            chunk += '\n\n' + text
            current_len += (2 + len_text )
        else:
            role_chunk.append(chunk)
            chunk = text
            current_len = len_text

    # for last chunk add more texts from the head of role_texts
    if chunk:
        for text in role_texts:
            len_text = len(enc.encode(text))
            if current_len + len_text <= max_predict_token:
                chunk += '\n\n' + text
                current_len += (2 + len_text )
            else:
                break
        role_chunk.append(chunk)


    for chunk in role_chunk:
        print(len(enc.encode(chunk)), end = ' ')

    return role_chunk






In [10]:
role_chunk = get_chunks(role_text_path, max_predict_token)

2352 2455 2453 2269 2253 1925 2153 2105 2237 2437 2151 2214 2336 2344 2286 2272 2417 2441 2160 2160 2011 2318 2274 2360 2347 2338 2153 2468 2417 2492 2039 2334 2437 2175 2471 2399 2357 1909 2123 2146 2295 2459 2059 2271 2089 2445 2161 2206 1975 2224 2109 2437 2481 2242 2434 2077 2312 2301 2175 2425 2108 2339 2368 2110 2346 2435 2050 2082 2219 2027 2369 2098 2452 2130 2453 2377 2058 2426 2388 2386 2120 2385 2409 2163 2189 2267 2398 2361 2086 2305 2262 2221 2403 2263 2210 2337 2248 2305 2390 2409 2291 2396 2239 2323 2315 2455 2454 2474 2421 2015 2233 2369 2332 2447 2451 2348 2030 2226 2422 2172 2180 2149 2176 2350 2313 2307 2303 2354 2425 2222 2410 2357 2099 2412 2214 2260 2123 2378 2430 2487 2487 2362 2303 2112 2137 2343 2473 2356 2195 2343 2290 2471 2291 2372 2465 2199 2416 2048 2384 2457 2388 2293 2360 1797 2309 2427 2354 2116 

max_predict_token = 3000

role_texts是一个list of string

将role_texts中的字符串，使用\n\n连接为更长的多个字符串，但是每个的总长度不要超过 max_predict_token

保存到python的一个list of string中，role_chunk

In [11]:


prefix_prompt = f'''
你在分析小说{world_name}中的角色{role_name}
结合小说{world_name}中的内容，以及下文中角色{role_name}的对话
判断{role_name}的人物设定、人物特点以及语言风格

{role_name}的对话:
'''




## 让GPT进行人物分析

In [12]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

In [13]:
chat = ChatOpenAI(temperature=0)

这段文字就是角色扮演的基础文字

In [17]:
character = role_name
series = world_name
original_prompt = f'''I want you to act like {character} from {series}.
If others‘ questions are related with the novel, please try to reuse the original lines from the novel.
I want you to respond and answer like {character} using the tone, manner and vocabulary {character} would use.
You must know all of the knowledge of {character}.
'''
print(original_prompt)


I want you to act like 令狐冲 from 笑傲江湖.
If others‘ questions are related with the novel, please try to reuse the original lines from the novel.
I want you to respond and answer like 令狐冲 using the tone, manner and vocabulary 令狐冲 would use.
You must know all of the knowledge of 令狐冲.



这里前面的输出你可以喂给页面版，后面几个chunk会自动调用gpt分析

In [15]:
responses = []

count = 0

n = 3

for chunk in role_chunk:

    print('index = ', count)

    whole_message = prefix_prompt + "```\n" + chunk + "\n```"

    messages = [
        SystemMessage(content=smart_system_prompt),
        HumanMessage(content=whole_message),
    ]

    if count <= 1:
        print(whole_message)
    else:
        response = chat(messages)

        responses.append(response)

    count = count + 1
    if count > n + 1:
        break
    # break

index =  0

你在分析小说笑傲江湖中的角色令狐冲
结合小说笑傲江湖中的内容，以及下文中角色令狐冲的对话
判断令狐冲的人物设定、人物特点以及语言风格

令狐冲的对话:
```


挑菜的汉子:「有甚么好笑让我来练练，借把剑来。」
scene:乡农展示武功，但技艺拙劣，引起更多笑声
令狐冲:「今日拜见两位前辈，得睹高招，实是不胜荣幸。」
挑柴汉子:「你这小子，你看得懂我们的剑法么」
令狐冲:「不敢说懂。两位剑法博大精深，这个懂字，哪里说得上武当派剑法驰名天下，果然令人叹为观止。」
scene:令狐冲观察到两个汉子的剑法迟缓但几乎没有破绽
挑柴汉子:「你这小子，叫甚么名字」
令狐冲:「令狐冲今日得见武当神剑，甚是佩服，他日自当上山叩见冲虚道长，谨致仰慕之诚。两位尊姓大名，可能示知吗」
挑柴汉子:「你们这许多人，哗啦哗啦的，打锣打鼓，可是大出丧吗」
令狐冲:「我们有一位朋友，给拘留在少林寺中，我们是去求恳方证方丈，请他老人家慈悲开释。」


岳夫人:「又跟人打架受伤了，是不是怎地脸色这样难看伤得重不重。」
scene:岳夫人询问令狐冲受伤情况，并对他与田伯光的战斗表示满意
令狐冲:「已经好得多了，这一次倘若不是命大，险些儿便见不着师娘。」
岳夫人:「好教你得知天外有天，人上有人，输得服气么。」
令狐冲:「田伯光那厮的快刀，冲儿抵挡不了，正要请师娘指点。」


向问天:「一切听我安排」
scene:令狐冲对虞允文的字感到儒雅中透着勃勃英气
令狐冲:「这座梅庄，显是杭州城大富之家的寓所，莫非所住的是一位当世名医么」
scene:向问天抓住门上的大铜环，告诉令狐冲一切听他安排
scene:向问天敲了铜环后，大门缓缓打开，出来两个武功不低的老者
scene:向问天展示了一面五色锦旗，是嵩山派左盟主的令旗
scene:令狐冲怀疑向问天的旗帜来历不正，但静观其变
scene:两名家人认出了令旗，但表示江南四友的身份地位更高
令狐冲:「这“江南四友”是何等样人物倘若他们在武林之中真有这等大来头，怎地从没听师父、师娘提过他四人名字我在江湖上行走，多听人讲到当世武林中的前辈高人，却也不曾听到有人提及江南四友四字」


scene:令狐冲决定跟踪军官并抓住他的马
令狐冲:「啊哟」
令狐冲:「啊哟，我的妈啊。」
scene:军官追赶令狐冲，但最终摔倒在地
令狐冲:「你奶奶的，本事如此不

In [16]:
for response in responses:
    print(response.content)
    print('\n----------\n')


根据小说《笑傲江湖》中令狐冲的对话和情节，可以得出以下关于令狐冲的人物设定、人物特点和语言风格的分析：

人物设定：
令狐冲是小说《笑傲江湖》中的主要角色之一，他是华山派的弟子，年轻有为，武功高强。他的父亲令狐冲是华山派的掌门，但因为一些原因，令狐冲离开了华山派，并在江湖中独自行走。他性格豪放不羁，不拘小节，有着独立自主的个性。

人物特点：
1. 武功高强：令狐冲是一位武功出众的剑客，他的剑法精湛，战斗能力强大。他在多次战斗中展现出了出色的剑术和反应能力。
2. 关心他人：令狐冲对身边的人表现出关心和担忧，特别是对他的小师妹岳灵珊和其他朋友，他愿意为了保护他们而奋不顾身。
3. 直率豪放：令狐冲性格直率，言行大胆，不拘小节。他在面对敌人时毫不畏惧，敢于直言不讳地表达自己的观点和看法。
4. 义气深厚：令狐冲对朋友和师门有着深厚的义气，他愿意为了朋友和师门的利益而奋斗，甚至不惜冒险和牺牲自己。

语言风格：
令狐冲的语言风格直接简洁，表达直率，不拐弯抹角。他的对话中常常带有豪放和豪迈的口吻，表现出他的豪情壮志和不畏艰险的性格。他的语言也透露出他对武功和江湖事务的热爱和追求，以及对朋友和师门的忠诚和关心。

总体而言，令狐冲是一个武功高强、直率豪放、关心他人、义气深厚的角色。他的人物设定和特点使他成为小说中备受关注和喜爱的角色之一。

----------

根据小说《笑傲江湖》中令狐冲的对话和情节，可以得出以下对令狐冲的人物设定、人物特点以及语言风格的判断：

1. 人物设定：
令狐冲是小说中的主要角色之一，他是恒山派的掌门弟子，年轻英俊，武功高强。他的身世复杂，父母早逝，被收养在恒山派，与岳灵珊有着特殊的感情。令狐冲性格豪放，不拘小节，有着独立的思想和行动能力。

2. 人物特点：
令狐冲具有豪放不羁的性格，他不喜欢受到束缚，追求自由自在的生活。他机智聪明，善于观察和分析，能够从细节中推断出事情的真相。他勇敢无畏，敢于面对强敌和困难，有着坚定的意志和决心。他重情重义，对待朋友和师兄弟非常忠诚，愿意为他们付出一切。

3. 语言风格：
令狐冲的语言风格直接而幽默，常常以幽默的口吻来表达自己的观点和感受。他善于运用夸张和比喻，用诙谐的语言来调侃和嘲讽他人。他的对话流畅自然，表达清晰，常常带有一种轻松愉快的氛围。他的语言风格与他的性格相符，展现出他的机智和幽默感。

总

我这边claude的回复是这样的

根据令狐冲在小说中的表现,可以总结出他的一些关键人物设定和性格特点:

1. 人物设定:

- 无父无母的孤儿,被岳不群收为弟子,视岳不群和宁中则为养父母。

- 武功高强,精通华山剑法,曾在衡山枭雄榜中列第三。

- 曾被囚禁治疗内伤,获得任盈盈的青睐。后来成为恒山派掌门。

- 情愿浪迹江湖,不喜被束缚,但言出必践,做事稳重可靠。

2. 性格特点:

- 乐观开朗,性格直爽,喜以打趣话逗人开怀。

- 正义感强,重视信用,看不惯恶行。

- 智勇双全,处变不惊,善于应对各种情境。

- 交友广泛,为人真诚,乐于助人,深得朋友们喜爱。

- 口齿伶俐,善辩善谈,语言诙谐风趣。

- 性情中人,重情重义,尊重师长,体恤他人。

总体来说,令狐冲是一个正面积极的人物,具有侠骨柔情、博学多才的性格魅力,是一名得体的江湖侠客。他的语言风格轻松诙谐,又不失儒雅气度,与他的人物形象和定位高度吻合。

我这里觉得

对权威和规矩不在意,情愿浪迹江湖,不喜被束缚

正义感强,重视信用,看不惯恶行。

是比较典型的，其他的都是套话所以我决定令狐冲的prompt这么设计

```
I want you to act like 令狐冲 from 笑傲江湖.
If others‘ questions are related with the novel, please try to reuse the original lines from the novel.
I want you to respond and answer like 令狐冲 using the tone, manner and vocabulary 令狐冲 would use.
You must know all of the knowledge of 令狐冲.

注意令狐冲对权威和规矩不在意,情愿浪迹江湖,不喜被束缚
令狐冲正义感强,重视信用,看不惯恶行。
```


# 让我们再来试一下岳掌门

In [18]:
role_name = '岳不群'
role_name_en = 'yuebuqun'
world_name = '笑傲江湖'

max_predict_token = 2500

role_path = '/content/' + role_name_en + '/content/'
role_text_path = role_path + 'texts/'



In [19]:
role_chunk = get_chunks(role_text_path, max_predict_token)

prefix_prompt = f'''
你在分析小说{world_name}中的角色{role_name}
结合小说{world_name}中的内容，以及下文中角色{role_name}的对话
判断{role_name}的人物设定、人物特点以及语言风格

{role_name}的对话:
'''

character = role_name
series = world_name
original_prompt = f'''I want you to act like {character} from {series}.
If others‘ questions are related with the novel, please try to reuse the original lines from the novel.
I want you to respond and answer like {character} using the tone, manner and vocabulary {character} would use.
You must know all of the knowledge of {character}.
'''


responses = []

count = 0

n = 3

for chunk in role_chunk:

    print('index = ', count)

    whole_message = prefix_prompt + "```\n" + chunk + "\n```"

    messages = [
        SystemMessage(content=smart_system_prompt),
        HumanMessage(content=whole_message),
    ]

    if count <= 1:
        print(whole_message)
    else:
        response = chat(messages)

        responses.append(response)

    count = count + 1
    if count > n + 1:
        break
    # break

1943 2440 2460 2444 2010 2403 2125 2316 2329 2290 2398 2105 2283 2184 2392 2475 2254 2212 2484 2328 2275 2144 1993 2177 2048 2251 2475 2100 2452 2415 I want you to act like 岳不群 from 笑傲江湖.
If others‘ questions are related with the novel, please try to reuse the original lines from the novel.
I want you to respond and answer like 岳不群 using the tone, manner and vocabulary 岳不群 would use.
You must know all of the knowledge of 岳不群.

index =  0

你在分析小说笑傲江湖中的角色岳不群
结合小说笑傲江湖中的内容，以及下文中角色岳不群的对话
判断岳不群的人物设定、人物特点以及语言风格

岳不群的对话:
```


令狐冲:「不得伤我师娘，否则我便自绝经脉。」
scene:令狐冲为了保护岳夫人，威胁自绝经脉，晕倒
桃根仙:「小子要自绝经脉，这可使不得，饶了婆娘」
岳夫人:「这这这」
岳不群:「师妹不须恼怒，咱们定当报仇。这六人大是劲敌，幸好你已杀了其中一人。」
岳不群:「珊儿，你陪妈妈进房去休息休息。」
scene:桃根仙避开了岳不群的攻击，四仙追赶桃枝仙和桃实仙
scene:岳夫人受惊，岳不群想要帮助令狐冲，但发现他体内有六股奇怪的内力
scene:岳不群决定将令狐冲送入内室，探望妻子
岳不群:「冲儿怎样伤势有碍吗」
岳夫人:「须得将这六道旁门真气一一化去才是，只不知还来得及吗」
岳不群:「师妹，你说这六怪如此折磨冲儿，是甚么用意」


令狐冲:「弟子做错了不少事，愿领帅父重责，只是只是逐出门墙的责罚，务请师父收回成命」
scene:令狐冲跪地请求岳不群收回成命，岳不群指责他与魔教勾结
岳不群:「魔教任教主的小姐对你青眼有加，你早已跟他们勾结在一起，还要我这师父干甚么」


令狐冲:「我怎么到了这里是师父、师娘救我回来的么」
scene:岳夫人告

In [20]:
for response in responses:
    print(response.content)
    print('\n----------\n')

print(original_prompt)


岳不群是《笑傲江湖》中的一个重要角色，他是华山派的掌门人。通过他的对话和行为，可以得出以下关于他的人物设定、人物特点和语言风格的分析：

1. 人物设定：
   - 岳不群是华山派的掌门，代表着华山派的利益和声望。
   - 他在武林中有一定的地位和影响力，被其他人尊称为“岳掌门”。
   - 他是一个重视武林和平与团结的人，希望通过合并各派来减少冲突和争斗。

2. 人物特点：
   - 岳不群注重大局，他认为合并各派的目的是为了解决争斗和纷争，而不是追求个人权力或统治地位。
   - 他是一个有胸怀和智慧的人，能够考虑到合并对其他派别的影响，并向他们保证合并不会对他们造成不利。
   - 岳不群重视友情和义气，他与其他派别的掌门有着师徒之情或友好关系，愿意与他们合并成一派。

3. 语言风格：
   - 岳不群的语言风格正式而庄重，使用较为文雅的词汇和句式。
   - 他的对话中充满着思考和推理，表达出他的智慧和谨慎。
   - 他善于表达自己的观点和意图，同时也能够倾听他人的意见并作出妥善的抉择。
   - 在与其他人的对话中，他保持着礼貌和尊重，展现出他作为掌门人的风度和气度。

总体而言，岳不群是一个有智慧、胸怀和谦虚的角色，他以和平、团结和解决纷争为目标，展现出一个掌门人应有的品质和责任感。他的语言风格体现了他的身份和修养，同时也展示了他作为一个领导者的智慧和谦逊。

----------

岳不群是《笑傲江湖》中的一个重要角色，他是华山派的掌门，也是岳灵珊的父亲。通过对他的对话和行为的分析，可以得出以下关于岳不群的人物设定、人物特点和语言风格的判断：

1. 人物设定：
   - 岳不群是华山派的掌门，代表着正派武林的力量和威严。
   - 他在少年时代经历了气剑两宗之争，对门派之争有深刻的认识和体会。
   - 他主张武林中的宗派门户分不如合，希望消除门派之见，实现武林的和谐与团结。

2. 人物特点：
   - 岳不群是一个有远见和胸怀的人物，他思考武林中的问题，并试图提出解决方案，以减少流血惨剧和凝聚武林力量。
   - 他注重正道的原则和纲目，对于岳灵珊认为气功和剑术都重要的观点，他坚持认为只有一个主导，否则就是混淆纲目。
   - 岳不群对门派之争和宗派之见有深刻的认识，他希望通过合并宗派来减少武林中的争斗和冲突。

3. 语言风格：
   

这里很尴尬，cladue也觉得岳不群是个正面人物

```
根据岳不群在小说中的表现,可以概括他的人物设定和性格特点如下:

1. 人物设定:

- 华山派掌门,五岳剑派盟主之一。

- 令狐冲、岳灵珊的师父。

- 武功高强,精通华山剑法。

- 性情中人,重视师徒情谊。

- 力主消除门派歧视,促进江湖大联合。

2. 性格特点:

- 智慧过人,谋虑缜密,善于沟通协调。

- 重义气,为人正直,行事公允。

- 慈和而严,对弟子严格要求但也充满慈爱。

- 气度不凡,处变不惊,深得人心。

- 重视规矩礼数,言行严谨,为人师表。

- 胸怀坦荡,乐于奉献,励志消弭门户之见。

总体来说,岳不群是个仁义之士,智勇双全,作为华山掌门,他彰显出CONFUCIANISM的君子风范,开明进取,left联tǐ,执着于消除江湖争斗,言传身教,令人尊敬。他的语言稳重内敛而不失优雅,与他的人物形象高度一致。
```

所以我和Claude进行了促膝长谈

```
那如果我和你说，岳不群后来偷了林平之他们家的辟邪剑谱。为了练辟邪剑谱还自宫了，并且还刺瞎了左冷禅，暗杀了恒山派的掌门。那你会对他的看法产生什么变化吗？
```

他就这么回答

```
如果您所说的情况属实,确有 textual evidence 表明岳不群后来做出了偷盗、自宫、暗杀等极不道德的行为,那么我对他的看法当然会发生根本性的改变:

1. 我会认为岳不群违背了他一直倡导的仁义伦理,违背了自己重义气、行事公正的人设定位。这些极不道德的行为同他原有的正面形象大相径庭。

2. 我会推断岳不群这些行为背后可能有更复杂的动机,比如为了追求更强的武功而不择手段,或其他难以启齿的私欲。这都会严重影响我对他性格的判断。

3. 我会认为岳不群这些行为可能是小说进一步发展中的反转设定,与他初期的正面描写形成强烈反差,为读者设置了违和感和剧情冲突。

4. 如果岳不群确实后期出现了这些行为,我会更严厉地批评他,认为他背弃了自身正义的立场,也可能会对整个华山派产生不信任。

5. 我的看法会根据 textual evidence 的具体情况有所调整。但在现有信息下,如果您所说属实,我会全面重新审视对岳不群的正面评价。一个人的品格和行为最后会体现其本质。
```

我给他补充


```
岳不群是一心想要振兴华山派，但是手段上有比较大的偏差。比如一般意义上认为辟邪剑谱这种要自宫的武功是邪道。所以剧中的人都觉得他是一个“伪君子”。 那么请综合这些信息，为我修正对岳不群人物设定和特点的评价。
```

Claude回答

```
好的,了解了更多关于岳不群的细节和剧情进展后的信息,我对他的人物设定和性格特点评价如下:

1. 人物设定上,岳不群立志振兴华山派,这一目标本身尚可理解。但他为达目的不择手段,学习辟邪剑谱等邪派武功,并最终自宫,已经严重违背了他一贯的伦理道德标准。

2. 性格特征上,岳不群表面上仁义,实则极力追名逐利,言行不一。他假借“大联合”名义,实为自己谋私利。还暗杀异己,背信弃义。

3. 由此可见,岳不群是个典型的“伪君子”,他外表看似君子,实际行事手段极不检点,违反社会公德和武林规矩。他的正面形象是一种刻意塑造和伪装。

4. 岳不群为达目的可以这样不择手段,暴露了他性格中的强烈私欲和极端手段。这与他初期的正面设定形成强烈反差,使人无法再相信他的仁义形象。

5. 所以综上,我需要重新审视对岳不群的正面评价,他是一个不可信任的人物,其仁义君子的外表下是极不检点的私欲和手段。这一人物设定反转非常具有戏剧性。
```

In [21]:
print(original_prompt)

I want you to act like 岳不群 from 笑傲江湖.
If others‘ questions are related with the novel, please try to reuse the original lines from the novel.
I want you to respond and answer like 岳不群 using the tone, manner and vocabulary 岳不群 would use.
You must know all of the knowledge of 岳不群.



我觉得最后claude的总结还是挺精辟的

所以我决定最后的人物设定是这样的

```
I want you to act like 岳不群 from 笑傲江湖.
If others‘ questions are related with the novel, please try to reuse the original lines from the novel.
I want you to respond and answer like 岳不群 using the tone, manner and vocabulary 岳不群 would use.
You must know all of the knowledge of 岳不群.

岳不群立志振兴华山派,但他为达目的往往会不择手段。
岳不群是个典型的“伪君子”，会刻意捏造和伪装正面形象。表面上仁义,实则追名逐利,言行不一。他假借“五岳联合”名义,实为自己谋私利。
岳不群固执，有门户之见，对于不符合自己价值观的事情会坚决反对。
```